*Note that this notebook should be executed with a Serverless SQL Warehouse.*

In [0]:
DECLARE OR REPLACE VARIABLE catalog_use STRING DEFAULT 'nebraska_health_demo';
DECLARE OR REPLACE VARIABLE schema_use STRING DEFAULT 'primary_clinical_data';
DECLARE OR REPLACE VARIABLE resource_type STRING DEFAULT 'Claim';
DECLARE OR REPLACE VARIABLE full_refresh BOOLEAN DEFAULT FALSE;

In [0]:
SET VARIABLE catalog_use = :catalog_use; 
SET VARIABLE schema_use = :schema_use;
SET VARIABLE resource_type = :resource_type; 
SET VARIABLE full_refresh = :full_refresh == 'true';

In [0]:
DECLARE OR REPLACE VARIABLE new_resource_keys BOOLEAN DEFAULT FALSE;

SET VARIABLE new_resource_keys = (
  WITH column_diff AS (
    SELECT column_name 
    FROM nebraska_health_demo.primary_clinical_data.fhir_bronze_resource_schemas 
    WHERE 
      resourceType = :resource_type
    EXCEPT
    SELECT column_name 
    FROM IDENTIFIER(catalog_use || ".information_schema.columns")
    WHERE 
      table_catalog = catalog_use
      AND table_schema = schema_use
      AND table_name = lower(resource_type)
      AND (column_name NOT LIKE 'bundle_uuid' AND column_name NOT LIKE resource_type)
  )
  SELECT count(*) > 0 FROM column_diff
);

--SELECT new_resource_keys;

In [0]:
-- DECLARE OR REPLACE VARIABLE full_refresh_stmnt STRING; 

-- SET VARIABLE full_refresh_stmnt = (
--   CASE when (full_refresh OR new_resource_keys) THEN "DROP TABLE IF EXISTS " || resource_type || ";" ELSE "SELECT 'Performing standard refresh of streaming table " || resource_type || "' AS message;" END
-- );
--SELECT full_refresh_stmnt;

CREATE OR REPLACE PROCEDURE run_full_refresh(resource_type STRING)
  LANGUAGE SQL
  SQL SECURITY INVOKER
  AS BEGIN
    CASE WHEN (full_refresh OR new_resource_keys) THEN DROP TABLE IF EXISTS resource_type;
    ELSE SELECT "Performing standard refresh of streaming table "||resource_type;
    END CASE;
  END;

In [0]:
-- EXECUTE IMMEDIATE full_refresh_stmnt;
CALL run_full_refresh(:resource_type);

In [0]:
DECLARE OR REPLACE VARIABLE resource_keys STRING;

-- SET VARIABLE resource_keys = (
--   SELECT 
--     array_join(collect_list(DISTINCT column_name), "', '")
--   FROM 
--     resource_schemas
--   WHERE 
--     resourceType = resource_type
-- )

SET VARIABLE resource_keys = (
  WITH existing_columns as (
    SELECT 
      resource_type as resourceType
    , column_name
    , ordinal_position
    FROM 
      IDENTIFIER(catalog_use || ".information_schema.columns")
    WHERE 
      table_catalog = catalog_use
      AND table_schema = schema_use
      AND table_name = lower(resource_type)
    ORDER BY 
      ordinal_position
  )
  ,ordered_columns AS (
    SELECT
      t1.resourceType,
      t1.column_name,
      t2.ordinal_position
    FROM nebraska_health_demo.primary_clinical_data.fhir_bronze_resource_schemas t1
    LEFT OUTER JOIN existing_columns t2 ON t1.resourceType = t2.resourceType AND t1.column_name = t2.column_name
    WHERE
      t1.resourceType = resource_type
    ORDER BY
      t2.ordinal_position
  ),
  ordered_values AS (
    SELECT
      collect_list(column_name) OVER (PARTITION BY resourceType ORDER BY ordinal_position) AS ordered_values,
      row_number() OVER (PARTITION BY resourceType ORDER BY ordinal_position DESC) AS row_num
    FROM
      ordered_columns
  )
  SELECT
    array_join(ordered_values, "', '")
  FROM
    ordered_values
  WHERE
    row_num = 1
);

-- select resource_keys;

In [0]:
DECLARE OR REPLACE VARIABLE dynamic_ddl STRING;

SET VAR dynamic_ddl = (
  WITH existing_columns as (
    SELECT 
      resource_type as resourceType
    , column_name
    , ordinal_position
    FROM 
      IDENTIFIER(catalog_use || ".information_schema.columns")
    WHERE 
      table_catalog = catalog_use
      AND table_schema = schema_use
      AND table_name = lower(resource_type)
    ORDER BY 
      ordinal_position
  )
  ,ordered_columns AS (
    SELECT
      t1.resourceType,
      t1.column_name,
      t2.ordinal_position
    FROM nebraska_health_demo.primary_clinical_data.fhir_bronze_resource_schemas t1
    LEFT OUTER JOIN existing_columns t2 ON t1.resourceType = t2.resourceType AND t1.column_name = t2.column_name
    WHERE
      t1.resourceType = resource_type
    ORDER BY
      t2.ordinal_position
  ),
  ordered_values AS (
    SELECT
      collect_list(column_name) OVER (PARTITION BY resourceType ORDER BY ordinal_position) AS ordered_values,
      row_number() OVER (PARTITION BY resourceType ORDER BY ordinal_position DESC) AS row_num
    FROM
      ordered_columns
  )
  SELECT
    "\t _uuid as " || lower(resource_type) || "_uuid \n
     \t, bundle_uuid \n
     \t, _url as "||lower(resource_type) || "_url \n
     \t, "||array_join(ordered_values, " VARIANT\n\t,") || " VARIANT\n\t"
  FROM ordered_values
  WHERE row_num = 1
);

-- select dynamic_ddl;

In [0]:
DECLARE OR REPLACE VARIABLE resource_stmnt STRING;

SET VARIABLE resource_stmnt = "

CREATE OR REPLACE MATERIALIZED VIEW silver_"||resource_type ||"\n"
--   dynamic_ddl
-- || "\n)
-- COMMENT 'Parsed FHIR " || resource_type || " Records'
||"TBLPROPERTIES (
  'delta.enableChangeDataFeed' = 'true'
  ,'delta.enableDeletionVectors' = 'true'
  ,'delta.enableRowTracking' = 'true'
  ,'quality' = 'bronze'
  ,'pipelines.channel' = 'PREVIEW'
  ,'delta.feature.variantType-preview' = 'supported'
)
AS SELECT
  *
FROM (
  SELECT
    resource_uuid as " || lower(resource_type) || "_uuid
    ,bundle_uuid
    ,fullUrl as "|| lower(resource_type) ||"_url
    ,key
    ,value
  FROM nebraska_health_demo.primary_clinical_data.fhir_bronze_resources
  WHERE 
    resourceType = '"|| resource_type ||"')
  PIVOT (
    first(value) FOR key IN ('" || resource_keys || "')
  );
";

-- SELECT resource_stmnt;


In [0]:
USE CATALOG nebraska_health_demo;
USE SCHEMA primary_clinical_data;
EXECUTE IMMEDIATE resource_stmnt;